In [1]:
# -*- coding: utf-8 -*- 
from lib import geoSearch
import json

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
GS = geoSearch.GeoSearch("elasticsearch-geo",9200)
from shapely.geometry import mapping, shape

In [3]:
#GS.test_kit('test_complete.csv')

In [4]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'elasticsearch-geo', 'port': 9200}])
from elasticsearch.client import IndicesClient
ic = IndicesClient(es)
def analyzedText(text):
    results = ic.analyze(index="montevideo", body=
    {
      "text": text, 
        "analyzer":"calle_analyzer"
    })
    return ' '.join([token['token'] for token in results['tokens']])

In [5]:
text = u"Limpiavidrios que está siempre en Ricaldoni y Av Italia (frente a la ancap). Vio a un ciego que intentaba cruzar, fue y lo ayudó en los dos tramos del cruce. Este tipo de gestos debemos imitar. Si pasas por ahí y te pide una moneda, dale. Es buen tipo. https://t.co/K6B0VPNfQ9"
print(analyzedText(text))

limpiavidrios siempre ricaldoni ricaldoni_y y_av av_italia italia italia_frente frente frente_a a_la la_ancap ancap vio vio_a a_un ciego intentaba cruzar fue_y y_lo ayudo en_los los_dos dos tramos tramos_del del_cruce cruce tipo gestos debemos imitar pasas ahi ahi_y y_te pide moneda dale buen tipo https t.co k6b0vpnfq9


In [6]:
for res in GS.multi_bool_cruces_calles(text, 150):
    obj = GS.get_result_object(res)
    print(GS.print_object(obj))

av italia # av dr americo ricaldoni ## 31.87831 
 -->   text_second_street
              normal  RICALDONI => 0.391
-->    text_first_street
              normal     ITALIA => 0.667
           variant_2  AV ITALIA => 1.000

av italia # av dr americo ricaldoni ## 31.87831 
 -->   text_second_street
              normal  RICALDONI => 0.391
-->    text_first_street
              normal     ITALIA => 0.667
           variant_2  AV ITALIA => 1.000

av italia # av dr americo ricaldoni ## 31.87831 
 -->   text_second_street
              normal  RICALDONI => 0.391
-->    text_first_street
              normal     ITALIA => 0.667
           variant_2  AV ITALIA => 1.000

av italia # av dr americo ricaldoni ## 31.87831 
 -->   text_second_street
              normal  RICALDONI => 0.391
-->    text_first_street
              normal     ITALIA => 0.667
           variant_2  AV ITALIA => 1.000

av italia # av dr americo ricaldoni ## 31.87831 
 -->   text_second_street
              normal  RICALDO

In [ ]:
matches = GS.complete_search(text,200)
for nombre, match in matches[:10]:
    if match:
        shape_1 = shape(match['objects'][0]['geometry'])
        print ('✔️ MATCH: {}\t/\t{}\t{}'.format(match['score'],match['field'],nombre))
print('\n')

In [ ]:
\n